In [1]:
import datetime as dt

import pandas as pd
import numpy as np
from lenskit.algorithms import item_knn, user_knn
from sklearn.model_selection import ParameterGrid

from recsys4daos.datasets import to_lenskit
from recsys4daos.model_selection import cvtt_open, explore_hparams
from recsys4daos.evaluation import test_with_hparams_lenskit

import paths

/home/daviddavo/recsys4daos/src/recsys4daos/model_selection.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Parameters

In [2]:
# Dataset config
ORG_NAME = 'Decentraland'
SPLITS_FREQ = 'W-THU'  # Split weekly
LAST_FOLDS = 20  # Use just last 10 splits
SPLITS_NORMALIZE = True

# Evaluation
K_RECOMMENDATIONS: list[int] = [1,3,5,10,15,100]
SPLITS_FREQ: str = 'W-THU' # split weekly
SPLITS_NORMALIZE = True # Wether or not to move everything to 00:00
LAST_FOLDS = 10 # Use just last 10 splits
LAST_FOLD_DATE_STR: str = None

# Search space config
WINDOW_SIZES = ['7d', '14d', '21d', '30d', '60d', '90d', '10YE']
ITEMKNN_Ks = [1,2,3,4,5,6,7,8,9,10,15]

OPTIM_METRIC = 'map@10'

In [3]:
# Parameters
EXECUTION_ID = "2024-09-04T10:00"
ORG_NAME = "dxDAO - xDXdao"
SPLITS_FREQ = "W-THU"
LAST_FOLDS = 10
SPLITS_NORMALIZE = True
LAST_FOLD_DATE_STR = "2022-05-05"


# Load the dataset

In [4]:
dfp = paths.load_proposals(ORG_NAME)
dfv = paths.load_votes(ORG_NAME)

print(dfp.info())
print(dfv.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 2356 non-null   object        
 1   author             2356 non-null   object        
 2   date               2356 non-null   datetime64[us]
 3   start              2356 non-null   datetime64[us]
 4   end                2323 non-null   datetime64[us]
 5   platform_proposal  2356 non-null   object        
dtypes: datetime64[us](3), object(3)
memory usage: 110.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8479 entries, 0 to 8478
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        8479 non-null   object        
 1   proposal  8479 non-null   object        
 2   voter     8479 non-null   object        
 3   date      8479 non-null   datetime64[us]
dtypes

In [5]:
df = to_lenskit(dfv)
df

,user,item,timestamp,rating
0,0x166cea845ade3f3b40ea68045d699dee5a645905,82f205b3-1f75-5da5-973a-fc2e324d8719,2019-07-14 19:50:09,1
1,0x166cea845ade3f3b40ea68045d699dee5a645905,b1f193e6-cf73-5a7a-85f3-a84d95c08b97,2019-07-14 19:51:24,1
2,0xe858a4bf603995a9156edbd25ff06269d997839e,b1f193e6-cf73-5a7a-85f3-a84d95c08b97,2019-07-14 22:24:38,1
3,0xe858a4bf603995a9156edbd25ff06269d997839e,398d685b-8c4f-5b7d-a7d6-86f1fc78efa4,2019-07-14 22:29:44,1
4,0xe858a4bf603995a9156edbd25ff06269d997839e,cb124ba8-a044-5790-8da9-43129a92c1f5,2019-07-14 22:31:43,1
...,...,...,...,...
8474,0x583acc79585d3cb195ea8125f6f80ad459b46313,783c82b4-a78e-58ee-972e-08a8d5677735,2023-04-08 07:38:59,1
8475,0x7e72cfd9a36517435dc1ca7f9451eccbc973111e,783c82b4-a78e-58ee-972e-08a8d5677735,2023-04-08 13:45:11,1
8476,0x3111327edd38890c3fe564afd96b4c73e8101747,783c82b4-a78e-58ee-972e-08a8d5677735,2023-04-08 18:13:23,1
8477,0x759a2169da1b826f795a00a9ab5f29f9ca39e48a,783c82b4-a78e-58ee-972e-08a8d5677735,2023-04-09 08:40:35,1


## Split in folds

In [6]:
all_folds = { f.end:f for f in cvtt_open(
    df, SPLITS_FREQ, dfp.reset_index(), remove_not_in_train_col='item', col_item='item', last_fold=LAST_FOLD_DATE_STR,
)}
last_folds_idx = list(all_folds.keys())[-LAST_FOLDS:]
last_folds_idx

[Timestamp('2022-03-03 00:00:00'),
 Timestamp('2022-03-10 00:00:00'),
 Timestamp('2022-03-17 00:00:00'),
 Timestamp('2022-03-24 00:00:00'),
 Timestamp('2022-03-31 00:00:00'),
 Timestamp('2022-04-07 00:00:00'),
 Timestamp('2022-04-14 00:00:00'),
 Timestamp('2022-04-21 00:00:00'),
 Timestamp('2022-04-28 00:00:00'),
 Timestamp('2022-05-05 00:00:00')]

# Item-based KNN

In [7]:
def testHParamsItemKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = item_knn.ItemItem(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsItemKNN(last_folds_idx[-1],5,'14d'))

/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
Numba is using threading layer omp - consider TBB


found 1 potential runtime problems - see https://boi.st/lkpy-perf


/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/lenskit/algorithms/item_knn.py:119: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  b = blocks[bi]


fold_t             2022-05-05 00:00:00
time_train                    5.562257
time_rec                      0.004441
open_proposals                      10
min_recs                             8
avg_recs                           9.0
precision@1                        0.0
precision@3                   0.166667
precision@5                        0.2
precision@10                       0.1
precision@15                  0.066667
precision@100                     0.01
ndcg@1                             0.0
ndcg@3                           0.125
ndcg@5                        0.221713
ndcg@10                       0.221713
ndcg@15                       0.221713
ndcg@100                      0.221713
map@1                              0.0
map@3                         0.166667
map@5                         0.266667
map@10                        0.266667
map@15                        0.266667
map@100                       0.266667
recall@1                           0.0
recall@3                 

## Exploring hparams

In [8]:
results = explore_hparams(
    testHParamsItemKNN, 
    ParameterGrid({
    'fold': last_folds_idx,
    'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
    'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('itemknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfi = pd.DataFrame(results)
mdfi

Restored checkpoint from ../.cache/dxDAO - xDXdao/hparams-itemknn_W-THU_normalize.pkl with 770 results


  0%|          | 0/770 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2022-03-03,1,7d,2022-03-03,0.001879,0.144894,16,6,13.000000,0.500000,...,0.611111,1.0,1.0,0.344444,0.755556,0.888889,0.888889,0.888889,0.888889,0.131315
1,2022-03-03,1,14d,2022-03-03,0.001218,0.008353,16,6,13.000000,0.333333,...,0.833333,1.0,1.0,0.350000,0.411111,0.411111,0.411111,0.411111,0.411111,0.129378
2,2022-03-03,1,21d,2022-03-03,0.001286,0.009810,16,6,13.428571,0.428571,...,0.809524,1.0,1.0,0.566667,0.558333,0.558333,0.558333,0.558333,0.558333,0.127850
3,2022-03-03,1,30d,2022-03-03,0.001772,0.011136,16,6,13.285714,0.285714,...,0.809524,1.0,1.0,0.350000,0.477778,0.544444,0.544444,0.544444,0.544444,0.127110
4,2022-03-03,1,60d,2022-03-03,0.005015,0.013675,16,6,13.285714,0.285714,...,0.809524,1.0,1.0,0.350000,0.477778,0.544444,0.544444,0.544444,0.544444,0.127180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2022-05-05,15,21d,2022-05-05,0.001125,0.003165,10,8,9.000000,0.000000,...,1.000000,1.0,1.0,0.000000,NaN,NaN,NaN,NaN,NaN,0.118569
766,2022-05-05,15,30d,2022-05-05,0.001537,0.003344,10,8,9.000000,0.000000,...,1.000000,1.0,1.0,0.000000,0.000000,NaN,NaN,NaN,NaN,0.111300
767,2022-05-05,15,60d,2022-05-05,0.008225,0.007448,10,8,9.500000,0.250000,...,1.000000,1.0,1.0,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.124579
768,2022-05-05,15,90d,2022-05-05,0.018284,0.007833,10,8,9.500000,0.250000,...,1.000000,1.0,1.0,0.333333,0.666667,0.666667,0.666667,0.666667,0.666667,0.124333


### Best overall hparams

In [9]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfi.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfi[mdfi['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train  avg_recs  precision@5  precision@10  \
window_size k                                                     
7d          10    0.003960  8.559127     0.229444      0.173042   
14d         5     0.002956  8.803571     0.244127      0.167540   
7d          15    0.002035  8.559127     0.229444      0.173042   
14d         6     0.002361  8.803571     0.244127      0.167540   
            8     0.001925  8.803571     0.244127      0.167540   
...                    ...       ...          ...           ...   
10YE        3     0.376218  9.124074     0.219136      0.163117   
60d         1     0.008274  9.164815     0.214321      0.168364   
10YE        2     0.375998  9.124074     0.215062      0.162006   
90d         2     0.018205  9.124074     0.223457      0.162130   
            1     0.018220  9.124074     0.218765      0.163241   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
7d          10       0.017999  0.411139  0.504240  0.517189  0.449356   
14d         5        0.017528  0.460639  0.537020  0.551272  0.467196   
7d          15       0.017999  0.409214  0.502315  0.515265  0.447042   
14d         6        0.017528  0.458335  0.534607  0.548858  0.462487   
            8        0.017528  0.456228  0.532500  0.546752  0.460661   
...                       ...       ...       ...       ...       ...   
10YE        3        0.017559  0.442781  0.552260  0.577948  0.345983   
60d         1        0.017559  0.420645  0.549990  0.564396  0.330242   
10YE        2        0.017559  0.421230  0.534831  0.561915  0.323837   
90d         2        0.017559  0.420547  0.526042  0.552899  0.321826   
            1        0.017559  0.407733  0.520413  0.545414  0.310159   

                  map@10   map@100  recall@5  recall@10  recall@100  \
window_size k                                                         
7d          10  0.524131  0.529786  0.625351   0.937500         1.0   
14d         5   0.521958  0.528308  0.703525   0.941468         1.0   
7d          15  0.521816  0.527471  0.625351   0.937500         1.0   
14d         6   0.517028  0.523378  0.703525   0.941468         1.0   
            8   0.515203  0.521553  0.703525   0.941468         1.0   
...                  ...       ...       ...        ...         ...   
10YE        3   0.411423  0.421560  0.629784   0.911728         1.0   
60d         1   0.404682  0.410775  0.611265   0.951852         1.0   
10YE        2   0.391646  0.402372  0.614969   0.908025         1.0   
90d         2   0.382383  0.393090  0.630401   0.909568         1.0   
            1   0.374304  0.384406  0.618673   0.915123         1.0   

                r-precision@5  r-precision@10  r-precision@100  
window_size k                                                   
7d          10       0.753819        0.753819         0.753819  
14d         5        0.748090        0.748090         0.748090  
7d          15       0.753819        0.753819         0.753819  
14d         6        0.727257        0.727257         0.727257  
            8        0.737674        0.737674         0.737674  
...                       ...             ...              ...  
10YE        3        0.612897        0.612897         0.612897  
60d         1        0.648214        0.648214         0.648214  
10YE        2        0.605952        0.605952         0.605952  
90d         2        0.632341        0.632341         0.632341  
            1        0.635317        0.635317         0.635317  

[77 rows x 17 columns]

Now let's see the behaviour in each fold

In [10]:
best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'itemknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/dxDAO - xDXdao/models/itemknn-best-avg_W-THU_normalize.parquet


/tmp/ipykernel_3821552/3951593685.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2022-03-03,7d,10,2022-03-03,0.001117,0.008061,16,6,13.000000,0.333333,0.333333,0.300000,0.183333,...,0.611111,1.0,1.0,0.350000,0.644444,0.777778,0.777778,0.777778,0.777778,0.128134
2022-03-10,7d,10,2022-03-10,0.001213,0.010838,16,7,11.875000,0.375000,0.208333,0.175000,0.225000,...,0.937500,1.0,1.0,0.444444,0.777778,0.645833,0.645833,0.645833,0.645833,0.127826
2022-03-17,7d,10,2022-03-17,0.001238,0.009082,15,6,10.333333,0.166667,0.055556,0.066667,0.066667,...,0.500000,1.0,1.0,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.131184
2022-03-24,7d,10,2022-03-24,0.002654,0.010312,14,6,10.857143,0.428571,0.238095,0.200000,0.185714,...,1.000000,1.0,1.0,0.833333,1.000000,0.812500,0.812500,0.812500,0.812500,0.134684
2022-03-31,7d,10,2022-03-31,0.005892,0.007030,9,4,7.400000,0.200000,0.333333,0.240000,0.200000,...,1.000000,1.0,1.0,0.500000,0.555556,0.555556,0.555556,0.555556,0.555556,0.131033
2022-04-07,7d,10,2022-04-07,0.005261,0.005689,11,5,7.500000,0.750000,0.666667,0.550000,0.300000,...,1.000000,1.0,1.0,0.277778,0.750000,0.833333,0.833333,0.833333,0.833333,0.125795
2022-04-14,7d,10,2022-04-14,0.001353,0.002117,17,4,4.000000,1.000000,0.333333,0.200000,0.100000,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.119270
2022-04-21,7d,10,2022-04-21,0.001764,0.004234,11,6,7.666667,0.333333,0.111111,0.133333,0.100000,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.126314
2022-04-28,7d,10,2022-04-28,0.001200,0.006613,11,8,8.400000,0.600000,0.466667,0.400000,0.280000,...,1.000000,1.0,1.0,0.316667,0.550000,0.683333,0.683333,0.683333,0.683333,0.127088


### Best hparams by fold

This are the parameters used to check which model is the best (validation)

In [11]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfi.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'itemknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/dxDAO - xDXdao/models/itemknn-best-val_W-THU_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2022-03-03,2,21d,0.001161,13.428571,0.257143,0.214286,0.024286,0.522058,0.644853,0.701576,0.449206,0.538747,0.562557,0.533333,0.809524,1.0,0.580000,0.580000,0.580000
2022-03-10,2,21d,0.001366,12.222222,0.200000,0.222222,0.024444,0.375524,0.556162,0.584464,0.358025,0.471484,0.488150,0.416667,0.907407,1.0,0.625000,0.625000,0.625000
2022-03-17,1,14d,0.001664,11.000000,0.085714,0.071429,0.011429,0.250000,0.320074,0.419516,0.242857,0.287075,0.320958,0.357143,0.571429,1.0,0.750000,0.750000,0.750000
2022-03-24,15,60d,0.008802,11.333333,0.200000,0.177778,0.017778,0.601748,0.739392,0.739392,0.541667,0.623898,0.623898,0.666667,1.000000,1.0,0.800000,0.800000,0.800000
2022-03-31,9,10YE,0.383094,7.200000,0.320000,0.200000,0.020000,0.701458,0.768322,0.768322,0.582222,0.648889,0.648889,0.866667,1.000000,1.0,0.541667,0.541667,0.541667
2022-04-07,1,14d,0.001197,8.200000,0.480000,0.260000,0.026000,0.694544,0.717720,0.717720,0.763333,0.796667,0.796667,0.950000,1.000000,1.0,0.875000,0.875000,0.875000
2022-04-14,15,90d,0.020592,4.000000,0.200000,0.100000,0.010000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000
2022-04-21,1,7d,0.005869,7.666667,0.133333,0.100000,0.010000,0.300000,0.366667,0.366667,0.444444,0.492063,0.492063,0.666667,1.000000,1.0,1.000000,1.000000,1.000000
2022-04-28,6,14d,0.001009,8.400000,0.440000,0.280000,0.028000,0.537439,0.637816,0.637816,0.558667,0.650333,0.650333,0.710000,1.000000,1.0,0.700000,0.700000,0.700000


In [12]:
best_hparams.tail(len(best_hparams)-1).describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.0,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2022-04-07 00:00:00,0.092639,0.012496,12.666667,5.888889,8.835802,0.541005,0.333010,0.251005,0.173492,...,0.942093,1.0,1.0,0.626235,0.749537,0.773148,0.773148,0.773148,0.773148,0.127623
min,2022-03-10 00:00:00,0.001009,0.004238,9.000000,4.000000,4.000000,0.250000,0.095238,0.085714,0.071429,...,0.571429,1.0,1.0,0.316667,0.541667,0.541667,0.541667,0.541667,0.541667,0.118476
25%,2022-03-24 00:00:00,0.001366,0.006919,11.000000,5.000000,7.666667,0.333333,0.250000,0.200000,0.100000,...,1.000000,1.0,1.0,0.416667,0.625000,0.666667,0.666667,0.666667,0.666667,0.126125
50%,2022-04-07 00:00:00,0.005869,0.011511,11.000000,6.000000,8.400000,0.555556,0.333333,0.200000,0.177778,...,1.000000,1.0,1.0,0.611111,0.750000,0.750000,0.750000,0.750000,0.750000,0.126978
75%,2022-04-21 00:00:00,0.020592,0.016798,15.000000,6.000000,11.000000,0.600000,0.400000,0.320000,0.222222,...,1.000000,1.0,1.0,0.750000,0.812500,0.875000,0.875000,0.875000,0.875000,0.131991
max,2022-05-05 00:00:00,0.410159,0.027489,17.000000,8.000000,12.222222,1.000000,0.600000,0.480000,0.280000,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.132655
std,NaN,0.172589,0.007468,2.872281,1.452966,2.521217,0.246788,0.146067,0.134393,0.073904,...,0.142332,0.0,0.0,0.265518,0.164840,0.160770,0.160770,0.160770,0.160770,0.004504


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb).

In [13]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfi.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'itemknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams[display_columns]

Saved dataframe into /home/daviddavo/recsys4daos/data/output/dxDAO - xDXdao/models/itemknn-best-test_W-THU_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2022-03-10,2,21d,0.001366,12.222222,0.200000,0.222222,0.024444,0.375524,0.556162,0.584464,0.358025,0.471484,0.488150,0.416667,0.907407,1.0,0.625000,0.625000,0.625000
2022-03-17,2,21d,0.004332,11.000000,0.057143,0.057143,0.011429,0.125000,0.195074,0.329384,0.100000,0.144218,0.191087,0.214286,0.428571,1.0,0.500000,0.500000,0.500000
2022-03-24,1,14d,0.003121,10.857143,0.228571,0.171429,0.018571,0.429425,0.498950,0.510672,0.442857,0.498413,0.510317,0.750000,0.964286,1.0,0.750000,0.750000,0.750000
2022-03-31,15,60d,0.008955,7.400000,0.320000,0.200000,0.020000,0.559966,0.624683,0.624683,0.425556,0.487460,0.487460,0.866667,1.000000,1.0,0.666667,0.666667,0.666667
2022-04-07,9,10YE,0.369823,8.666667,0.366667,0.250000,0.025000,0.520146,0.653918,0.653918,0.417593,0.506680,0.506680,0.666667,1.000000,1.0,0.604167,0.604167,0.604167
2022-04-14,1,14d,0.001190,4.000000,0.200000,0.100000,0.010000,0.630930,0.630930,0.630930,0.500000,0.500000,0.500000,1.000000,1.000000,1.0,NaN,NaN,NaN
2022-04-21,15,90d,0.019941,9.000000,0.120000,0.100000,0.010000,0.312321,0.445655,0.445655,0.216667,0.273810,0.273810,0.600000,1.000000,1.0,NaN,NaN,NaN
2022-04-28,1,7d,0.001633,8.400000,0.360000,0.280000,0.028000,0.343892,0.514733,0.514733,0.342000,0.478905,0.478905,0.510000,1.000000,1.0,0.775000,0.775000,0.775000
2022-05-05,6,14d,0.005944,9.000000,0.200000,0.100000,0.010000,0.221713,0.221713,0.221713,0.266667,0.266667,0.266667,1.000000,1.000000,1.0,NaN,NaN,NaN


In [14]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.0,9.0,9.000000,6.000000,6.000000,6.000000,6.000000,6.000000,9.000000
mean,2022-04-07 00:00:00,0.046256,0.012520,12.666667,5.888889,8.949559,0.194356,0.269136,0.228042,0.164533,...,0.922252,1.0,1.0,0.282407,0.609722,0.653472,0.653472,0.653472,0.653472,0.123563
min,2022-03-10 00:00:00,0.001190,0.002404,9.000000,4.000000,4.000000,0.000000,0.047619,0.057143,0.057143,...,0.428571,1.0,1.0,0.000000,0.500000,0.500000,0.500000,0.500000,0.500000,0.113876
25%,2022-03-24 00:00:00,0.001633,0.006609,11.000000,5.000000,8.400000,0.000000,0.166667,0.200000,0.100000,...,0.964286,1.0,1.0,0.000000,0.550000,0.609375,0.609375,0.609375,0.609375,0.119978
50%,2022-04-07 00:00:00,0.004332,0.012440,11.000000,6.000000,9.000000,0.200000,0.333333,0.200000,0.171429,...,1.000000,1.0,1.0,0.291667,0.600000,0.645833,0.645833,0.645833,0.645833,0.126572
75%,2022-04-21 00:00:00,0.008955,0.013102,15.000000,6.000000,10.857143,0.333333,0.333333,0.320000,0.222222,...,1.000000,1.0,1.0,0.500000,0.656250,0.729167,0.729167,0.729167,0.729167,0.126866
max,2022-05-05 00:00:00,0.369823,0.034532,17.000000,8.000000,12.222222,0.444444,0.400000,0.366667,0.280000,...,1.000000,1.0,1.0,0.583333,0.750000,0.775000,0.775000,0.775000,0.775000,0.128925
std,NaN,0.121481,0.009481,2.872281,1.452966,2.393739,0.177748,0.123550,0.105074,0.078499,...,0.187742,0.0,0.0,0.235498,0.090587,0.101051,0.101051,0.101051,0.101051,0.005449


# User-based KNN

In [15]:
def testHParamsUserKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = user_knn.UserUser(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsUserKNN(last_folds_idx[-1],5,'14d'))

fold_t             2022-05-05 00:00:00
time_train                    0.591782
time_rec                      0.833733
open_proposals                      10
min_recs                             8
avg_recs                           9.0
precision@1                        0.0
precision@3                   0.166667
precision@5                        0.1
precision@10                       0.1
precision@15                  0.066667
precision@100                     0.01
ndcg@1                             0.0
ndcg@3                        0.157732
ndcg@5                        0.157732
ndcg@10                       0.246784
ndcg@15                       0.246784
ndcg@100                      0.246784
map@1                              0.0
map@3                             0.25
map@5                             0.25
map@10                        0.333333
map@15                        0.333333
map@100                       0.333333
recall@1                           0.0
recall@3                 

## Exploring hparams

In [16]:
results = explore_hparams(
    testHParamsUserKNN, 
    ParameterGrid({
        'fold': last_folds_idx,
        'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
        'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('userknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfu = pd.DataFrame(results)
mdfu

Restored checkpoint from ../.cache/dxDAO - xDXdao/hparams-userknn_W-THU_normalize.pkl with 770 results


  0%|          | 0/770 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2022-03-03,1,7d,2022-03-03,0.000432,0.005378,16,6,13.000000,0.333333,...,0.611111,1.0,1.0,0.35,0.644444,0.777778,0.777778,0.777778,0.777778,0.129018
1,2022-03-03,1,14d,2022-03-03,0.000387,0.004713,16,6,13.000000,0.333333,...,0.611111,1.0,1.0,0.35,0.477778,0.477778,0.477778,0.477778,0.477778,0.126167
2,2022-03-03,1,21d,2022-03-03,0.000392,0.005526,16,6,13.428571,0.285714,...,0.809524,1.0,1.0,0.35,0.477778,0.477778,0.477778,0.477778,0.477778,0.125067
3,2022-03-03,1,30d,2022-03-03,0.000421,0.005613,16,6,13.285714,0.285714,...,0.809524,1.0,1.0,0.35,0.477778,0.477778,0.477778,0.477778,0.477778,0.125135
4,2022-03-03,1,60d,2022-03-03,0.000498,0.005950,16,6,13.285714,0.285714,...,0.761905,1.0,1.0,0.35,0.477778,0.477778,0.477778,0.477778,0.477778,0.124761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2022-05-05,15,21d,2022-05-05,0.000414,0.002251,10,8,9.000000,0.000000,...,1.000000,1.0,1.0,0.00,NaN,NaN,NaN,NaN,NaN,0.121914
766,2022-05-05,15,30d,2022-05-05,0.000470,0.002200,10,8,9.000000,0.000000,...,1.000000,1.0,1.0,0.00,NaN,NaN,NaN,NaN,NaN,0.121711
767,2022-05-05,15,60d,2022-05-05,0.000824,0.004080,10,8,9.500000,0.000000,...,1.000000,1.0,1.0,0.00,0.333333,0.333333,0.333333,0.333333,0.333333,0.122717
768,2022-05-05,15,90d,2022-05-05,0.000827,0.004107,10,8,9.500000,0.000000,...,1.000000,1.0,1.0,0.00,0.666667,0.666667,0.666667,0.666667,0.666667,0.122250


### Best overall hparams

In [17]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfu.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfu[mdfu['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train  avg_recs  precision@5  precision@10  \
window_size k                                                     
14d         1     0.000382  8.803571     0.228968      0.169127   
7d          3     0.000337  8.559127     0.234762      0.170265   
            5     0.000360  8.559127     0.234762      0.170265   
            15    0.000358  8.559127     0.234762      0.170265   
            10    0.000364  8.559127     0.234762      0.170265   
...                    ...       ...          ...           ...   
60d         1     0.000592  9.164815     0.215864      0.163364   
            3     0.000577  9.164815     0.205370      0.162253   
            2     0.000578  9.164815     0.196111      0.162253   
10YE        2     0.003424  9.124074     0.186481      0.162253   
            3     0.003469  9.124074     0.195000      0.162253   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
14d         1        0.017528  0.463207  0.554027  0.566976  0.467812   
7d          3        0.017999  0.426153  0.510218  0.526219  0.469362   
            5        0.017999  0.425483  0.509548  0.525549  0.469031   
            15       0.017999  0.425483  0.509548  0.525549  0.469031   
            10       0.017999  0.425483  0.509548  0.525549  0.469031   
...                       ...       ...       ...       ...       ...   
60d         1        0.017559  0.455811  0.580365  0.604867  0.384450   
            3        0.017559  0.439670  0.573860  0.600210  0.378915   
            2        0.017559  0.430499  0.572360  0.598710  0.369578   
10YE        2        0.017559  0.421995  0.571078  0.597993  0.366723   
            3        0.017559  0.427540  0.571108  0.598022  0.371003   

                  map@10   map@100  recall@5  recall@10  recall@100  \
window_size k                                                         
14d         1   0.540598  0.545625  0.617778   0.945437         1.0   
7d          3   0.534118  0.542193  0.625714   0.927083         1.0   
            5   0.533788  0.541862  0.625714   0.927083         1.0   
            15  0.533788  0.541862  0.625714   0.927083         1.0   
            10  0.533788  0.541862  0.625714   0.927083         1.0   
...                  ...       ...       ...        ...         ...   
60d         1   0.455972  0.465845  0.592747   0.917593         1.0   
            3   0.453208  0.464019  0.559414   0.912037         1.0   
            2   0.450595  0.461407  0.545525   0.912037         1.0   
10YE        2   0.449355  0.459585  0.514969   0.909259         1.0   
            3   0.448438  0.458668  0.525154   0.909259         1.0   

                r-precision@5  r-precision@10  r-precision@100  
window_size k                                                   
14d         1        0.677315        0.761979         0.761979  
7d          3        0.791146        0.791146         0.791146  
            5        0.791146        0.791146         0.791146  
            15       0.791146        0.791146         0.791146  
            10       0.791146        0.791146         0.791146  
...                       ...             ...              ...  
60d         1        0.697817        0.697817         0.697817  
            3        0.749405        0.749405         0.749405  
            2        0.756349        0.756349         0.756349  
10YE        2        0.748413        0.748413         0.748413  
            3        0.668750        0.668750         0.668750  

[77 rows x 17 columns]

In [18]:
best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'userknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/dxDAO - xDXdao/models/userknn-best-avg_W-THU_normalize.parquet


/tmp/ipykernel_3821552/3337470013.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2022-03-03,14d,1,2022-03-03,0.000387,0.004713,16,6,13.000000,0.333333,0.333333,0.233333,0.183333,...,0.611111,1.0,1.0,0.350000,0.477778,0.477778,0.477778,0.477778,0.477778,0.126167
2022-03-10,14d,1,2022-03-10,0.000377,0.006384,16,7,11.875000,0.375000,0.291667,0.275000,0.225000,...,0.937500,1.0,1.0,0.444444,0.645833,0.708333,0.708333,0.708333,0.708333,0.133332
2022-03-17,14d,1,2022-03-17,0.000408,0.005775,15,6,11.000000,0.142857,0.095238,0.085714,0.071429,...,0.571429,1.0,1.0,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.132649
2022-03-24,14d,1,2022-03-24,0.000415,0.006072,14,6,10.857143,0.428571,0.285714,0.200000,0.185714,...,1.000000,1.0,1.0,0.833333,0.687500,0.687500,0.687500,0.687500,0.687500,0.132817
2022-03-31,14d,1,2022-03-31,0.000387,0.004626,9,4,7.400000,0.400000,0.466667,0.320000,0.200000,...,1.000000,1.0,1.0,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.129967
2022-04-07,14d,1,2022-04-07,0.000378,0.004389,11,5,8.200000,0.600000,0.466667,0.400000,0.260000,...,1.000000,1.0,1.0,0.277778,0.666667,0.750000,0.750000,0.750000,0.750000,0.132500
2022-04-14,14d,1,2022-04-14,0.000384,0.001301,17,4,4.000000,1.000000,0.333333,0.200000,0.100000,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.123609
2022-04-21,14d,1,2022-04-21,0.000398,0.003514,11,6,8.500000,0.250000,0.083333,0.100000,0.100000,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.130360
2022-04-28,14d,1,2022-04-28,0.000310,0.004160,11,8,8.400000,0.600000,0.600000,0.480000,0.280000,...,1.000000,1.0,1.0,0.316667,0.600000,0.700000,0.700000,0.700000,0.700000,0.130837


### Best hparams by fold

In [19]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfu.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'userknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/dxDAO - xDXdao/models/userknn-best-val_W-THU_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2022-03-03,8,7d,0.000348,13.000000,0.300000,0.183333,0.023333,0.395052,0.449052,0.543023,0.444444,0.473380,0.512269,0.444444,0.611111,1.0,0.888889,0.888889,0.888889
2022-03-10,1,14d,0.000377,11.875000,0.275000,0.225000,0.023750,0.379511,0.497622,0.514726,0.392708,0.484053,0.495416,0.583333,0.937500,1.0,0.708333,0.708333,0.708333
2022-03-17,1,10YE,0.003303,11.250000,0.050000,0.075000,0.011250,0.201643,0.353681,0.452287,0.187500,0.259673,0.288633,0.187500,0.625000,1.0,0.750000,0.750000,0.750000
2022-03-24,2,7d,0.000351,10.857143,0.171429,0.185714,0.018571,0.422117,0.551719,0.551719,0.508929,0.604521,0.604521,0.607143,1.000000,1.0,0.812500,0.812500,0.812500
2022-03-31,1,10YE,0.003460,7.200000,0.320000,0.200000,0.020000,0.776368,0.841085,0.841085,0.710000,0.771905,0.771905,0.866667,1.000000,1.0,0.833333,0.833333,0.833333
2022-04-07,10,7d,0.000346,7.500000,0.550000,0.300000,0.030000,0.492638,0.548194,0.548194,0.721875,0.757589,0.757589,0.750000,1.000000,1.0,0.916667,0.916667,0.916667
2022-04-14,2,90d,0.000716,4.000000,0.200000,0.100000,0.010000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000
2022-04-21,7,7d,0.000313,7.666667,0.133333,0.100000,0.010000,0.286135,0.352802,0.352802,0.416667,0.464286,0.464286,0.666667,1.000000,1.0,1.000000,1.000000,1.000000
2022-04-28,1,14d,0.000310,8.400000,0.480000,0.280000,0.028000,0.657080,0.698088,0.698088,0.688667,0.747000,0.747000,0.910000,1.000000,1.0,0.700000,0.700000,0.700000


In [20]:
best_hparams.tail(len(best_hparams)-1).describe()[display_columns]

,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.0,9.000000,9.000000,9.000000
mean,0.001418,8.694312,0.258862,0.179524,0.018508,0.508298,0.599775,0.612632,0.549532,0.611468,0.615949,0.665331,0.951389,1.0,0.820833,0.820833,0.820833
min,0.000310,4.000000,0.050000,0.075000,0.010000,0.201643,0.352802,0.352802,0.187500,0.259673,0.288633,0.187500,0.625000,1.0,0.666667,0.666667,0.666667
25%,0.000346,7.500000,0.150000,0.100000,0.011250,0.359187,0.497622,0.514726,0.392708,0.464286,0.464286,0.583333,1.000000,1.0,0.708333,0.708333,0.708333
50%,0.000377,8.400000,0.200000,0.185714,0.018571,0.422117,0.551719,0.551719,0.508929,0.604521,0.604521,0.666667,1.000000,1.0,0.812500,0.812500,0.812500
75%,0.003303,10.857143,0.320000,0.225000,0.023750,0.657080,0.698088,0.698088,0.710000,0.757589,0.757589,0.866667,1.000000,1.0,0.916667,0.916667,0.916667
max,0.003589,11.875000,0.550000,0.300000,0.030000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.000000
std,0.001531,2.470169,0.165832,0.080297,0.007596,0.256333,0.214203,0.201111,0.251405,0.228321,0.222080,0.254911,0.124129,0.0,0.127305,0.127305,0.127305


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb)

In [21]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfu.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'userknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/dxDAO - xDXdao/models/userknn-best-test_W-THU_normalize.parquet


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,k,window_size,,,,,,,,,,,,,,,,,,,,,
2022-03-10,8,7d,2022-03-10,0.000322,0.006283,16,7,11.875000,0.250000,0.208333,0.200000,0.200000,...,0.843750,1.0,1.0,0.500000,0.777778,0.645833,0.645833,0.645833,0.645833,0.132398
2022-03-17,1,14d,2022-03-17,0.000408,0.005775,15,6,11.000000,0.142857,0.095238,0.085714,0.071429,...,0.571429,1.0,1.0,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.132649
2022-03-24,1,10YE,2022-03-24,0.003442,0.011823,14,6,11.333333,0.222222,0.222222,0.177778,0.177778,...,1.000000,1.0,1.0,0.750000,0.583333,0.583333,0.583333,0.583333,0.583333,0.131762
2022-03-31,2,7d,2022-03-31,0.000354,0.004359,9,4,7.400000,0.200000,0.266667,0.320000,0.200000,...,1.000000,1.0,1.0,0.500000,0.666667,0.666667,0.666667,0.666667,0.666667,0.130463
2022-04-07,1,10YE,2022-04-07,0.003409,0.009092,11,5,8.666667,0.666667,0.444444,0.366667,0.250000,...,1.000000,1.0,1.0,0.333333,0.625000,0.750000,0.750000,0.750000,0.750000,0.130711
2022-04-14,10,7d,2022-04-14,0.000387,0.001274,17,4,4.000000,1.000000,0.333333,0.200000,0.100000,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.123316
2022-04-21,2,90d,2022-04-21,0.000702,0.005103,11,6,9.000000,0.000000,0.066667,0.120000,0.100000,...,1.000000,1.0,1.0,0.000000,NaN,NaN,NaN,NaN,NaN,0.121677
2022-04-28,7,7d,2022-04-28,0.000300,0.004105,11,8,8.400000,0.600000,0.266667,0.400000,0.280000,...,1.000000,1.0,1.0,0.316667,0.383333,0.600000,0.600000,0.600000,0.600000,0.131194
2022-05-05,1,14d,2022-05-05,0.000385,0.002082,10,8,9.000000,0.000000,0.000000,0.000000,0.100000,...,1.000000,1.0,1.0,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.106835


In [22]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.0,9.0,9.000000,8.000000,8.000000,7.000000,7.000000,7.000000,9.000000
mean,2022-04-07 00:00:00,0.001079,0.005544,12.666667,6.0,8.963889,0.342416,0.211508,0.207795,0.164356,...,0.935020,1.0,1.0,0.433333,0.567014,0.593229,0.677976,0.677976,0.677976,0.126778
min,2022-03-10 00:00:00,0.000300,0.001274,9.000000,4.0,4.000000,0.000000,0.000000,0.000000,0.071429,...,0.571429,1.0,1.0,0.000000,0.000000,0.000000,0.500000,0.500000,0.500000,0.106835
25%,2022-03-24 00:00:00,0.000354,0.004105,11.000000,5.0,8.400000,0.142857,0.095238,0.120000,0.100000,...,1.000000,1.0,1.0,0.316667,0.470833,0.562500,0.591667,0.591667,0.591667,0.123316
50%,2022-04-07 00:00:00,0.000387,0.005103,11.000000,6.0,9.000000,0.222222,0.222222,0.200000,0.177778,...,1.000000,1.0,1.0,0.500000,0.604167,0.622917,0.645833,0.645833,0.645833,0.130711
75%,2022-04-21 00:00:00,0.000702,0.006283,15.000000,7.0,11.000000,0.600000,0.266667,0.320000,0.200000,...,1.000000,1.0,1.0,0.500000,0.694444,0.687500,0.708333,0.708333,0.708333,0.131762
max,2022-05-05 00:00:00,0.003442,0.011823,17.000000,8.0,11.875000,1.000000,0.444444,0.400000,0.280000,...,1.000000,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.132649
std,NaN,0.001336,0.003288,2.872281,1.5,2.390854,0.339392,0.138989,0.133035,0.074574,...,0.145811,0.0,0.0,0.322856,0.294034,0.282579,0.161638,0.161638,0.161638,0.008479
